In [1]:
import pandas as pd 
import numpy as np

In [2]:
restaurants = pd.read_csv('restaurants_data.csv')
original = pd.read_csv('original_restaurants.csv')

In [17]:
# filter that are in original but not in restaurants
user_restaurants = original[~original['Place ID'].isin(restaurants['Place ID'])]

In [18]:
user_places = user_restaurants['Place ID'].tolist()

In [19]:
import requests

def get_place_coordinates(place_id, api_key):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        'place_id': place_id,
        'fields': 'geometry',
        'key': api_key
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        result = response.json().get('result', {})
        location = result.get('geometry', {}).get('location', {})
        latitude = location.get('lat', 'No latitude available')
        longitude = location.get('lng', 'No longitude available')
        return latitude, longitude
    else:
        return f"Error: {response.status_code}", None


/tmp/ipykernel_98077/3421209239.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_restaurants.loc[user_restaurants['Place ID'] == place_id, 'Latitude'] = latitude
/tmp/ipykernel_98077/3421209239.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_restaurants.loc[user_restaurants['Place ID'] == place_id, 'Longitude'] = longitude


In [ ]:
# get the coordinates for each place
for place_id in user_places:
    latitude, longitude = get_place_coordinates(place_id, API_KEY)
    user_restaurants.loc[user_restaurants['Place ID'] == place_id, 'Latitude'] = latitude
    user_restaurants.loc[user_restaurants['Place ID'] == place_id, 'Longitude'] = longitude


In [7]:
import pandas as pd 
user_restaurants = pd.read_csv('user_restaurants.csv')
restaurants.columns

Index(['Place ID', 'Name', 'Summary', 'Price Level', 'Address', 'Latitude',
       'Longitude', 'Overall Rating', 'Opening Hours', 'Total Reviews',
       'Reviews', 'Review Ratings'],
      dtype='object')

In [35]:
# create a user table using random numbers with the following columns: user_id, place_id, user_rating, user_review, visit_date, visit_time and merge with the user_restaurants table randomly
import random
import datetime
import numpy as np 

def random_time_between(start_hour=10, end_hour=24):
    # Convert the start and end hours to seconds from midnight
    start_time_in_seconds = start_hour * 3600  # 10 AM in seconds
    end_time_in_seconds = end_hour * 3600      # 12 AM (midnight) in seconds
    
    # Generate random seconds between the start and end time
    random_seconds = random.randint(0, end_time_in_seconds - start_time_in_seconds)
    
    # Calculate the random time by adding the random seconds to the start time
    random_time = (datetime.datetime.combine(datetime.date.today(), datetime.time(hour=start_hour)) + 
                   datetime.timedelta(seconds=random_seconds)).time()
    
    # Return the time formatted as HH:MM:SS (24-hour format)
    return random_time.strftime("%H:%M:%S")

# Generate random visit times for 100 entries between 10 AM and 12 AM
# Generate random visit times for 100 entries between 8 AM and 12 AM
# visit_times = [random_time_between() for _ in range(100)]

user_ids = np.arange(1, 101)

# select random place ids from the user_restaurants table
user_place_ids = user_restaurants['Place ID'].sample(100).tolist()
user_ratings = np.random.randint(1, 6, 100)
visit_dates = [datetime.datetime.now() - datetime.timedelta(days=random.randint(1, 365)) for _ in range(100)]
visit_times = [random_time_between() for _ in range(100)]

user_data = {
    'user_id': user_ids,
    'place_id': user_place_ids,
    'user_rating': user_ratings,
    # 'user_review': user_reviews,
    'visit_date': visit_dates,
    'visit_time': visit_times
}

user_table = pd.DataFrame(user_data)

user_table

user_restaurants_ratings = user_restaurants.merge(user_table, left_on='Place ID', right_on='place_id', how='right')

# user_restaurants.to_csv('user_ratings_restaurants.csv', index=False)


In [39]:
# user_restaurants_ratings.user_rating.unique()

user_restaurants_ratings


,Place ID,Name,Summary,Price Level,Address,Latitude,Longitude,Overall Rating,Opening Hours,Total Reviews,Reviews,Review Ratings,user_id,place_id,user_rating,visit_date,visit_time
0,ChIJG_cDGdtRqEcR4ZXXlDTg-uI,Charlotte & Fritz,"Upscale fine-dining venue with chandeliers, se...",4.0,"Charlottenstraße 49, Berlin",52.515253,13.390753,4.3,"Monday: Closed, Tuesday: Closed, Wednesday: Cl...",159,"[""There are different high-class restaurants. ...","[5, 5, 5]",1,ChIJG_cDGdtRqEcR4ZXXlDTg-uI,2,2024-03-28 20:11:26.579885,12:03:10
1,ChIJTYcTLspRqEcRM50HDuvMReE,Vox Restaurant & Bar,Elegant restaurant & bar in the Grand Hyatt fe...,4.0,"Marlene-Dietrich-Platz 2, Berlin",52.507941,13.372730,4.4,"Monday: 6:30 – 10:00 PM, Tuesday: 6:30 – 10:00...",566,['Vox Restaurant Berlin offers a unique dining...,"[5, 5, 5]",2,ChIJTYcTLspRqEcRM50HDuvMReE,1,2024-04-20 20:11:26.579898,11:18:12
2,ChIJ4WacgF5OqEcREmhv4K5Cm9c,Lemongrass,"Pad Thai, coconut-based curries & rice dishes ...",2.0,"Simon-Dach-Straße 2, Berlin",52.512542,13.457129,4.2,"Monday: 12:00 – 10:00 PM, Tuesday: 12:00 – 10:...",1452,['Had a great dinner in this restaurant! All d...,"[5, 4, 1]",3,ChIJ4WacgF5OqEcREmhv4K5Cm9c,5,2023-09-24 20:11:26.579901,11:30:46
3,ChIJt6TDNDJOqEcRgkSFkyYYhfk,Oranienberger,"Late-night bar serving coffee, tea & cocktails...",2.0,"Oranienplatz 15, Berlin",52.502386,13.416971,4.4,"Monday: 12:00 PM – 12:00 AM, Tuesday: 12:00 PM...",294,"['So, we came here on a pub crawl. It was empt...","[3, 5, 5]",4,ChIJt6TDNDJOqEcRgkSFkyYYhfk,3,2023-09-14 20:11:26.579903,14:29:04
4,ChIJm3_SgO9RqEcRPsCFGSCiw4U,Cafe Ribo,Unpretentious weekdays-only outfit serving dum...,NaN,"Ackerstraße 157, Berlin",52.530891,13.396628,4.7,"Monday: 11:30 AM – 8:00 PM, Tuesday: 11:30 AM ...",66,"[""Such a pleasant surprise, in the midst of mi...","[5, 5, 5]",5,ChIJm3_SgO9RqEcRPsCFGSCiw4U,4,2024-08-22 20:11:26.579905,13:37:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ChIJP6_0bshPqEcR0W9r9JrC_5c,Ristorante Masaniello,Casual Italian restaurant with a patio & rusti...,2.0,"Hasenheide 20, Berlin",52.487945,13.419131,4.3,"Monday: 5:00 PM – 12:00 AM, Tuesday: 5:00 PM –...",2234,"['We took pizza take away, but we waited insid...","[5, 5, 4]",96,ChIJP6_0bshPqEcR0W9r9JrC_5c,3,2024-02-19 20:11:26.580091,23:13:07
96,ChIJhYcCg_RRqEcR2BHrK-3C_nI,Café & Bistro freysinn,"Salads, soups & pasta dishes, plus vegetarian ...",2.0,"Jasmunder Straße 5, Berlin",52.539842,13.388170,4.3,"Monday: 8:00 AM – 3:00 PM, Tuesday: 8:00 AM – ...",194,"['Delicious food, you can go every day and you...","[5, 2, 5]",97,ChIJhYcCg_RRqEcR2BHrK-3C_nI,4,2024-04-02 20:11:26.580094,11:34:25
97,ChIJrcPw105QqEcR_0-GmjViOMk,Restaurant Boccacelli am Winterfeldtplatz Berlin,"Large pizzas, homemade pasta & classic Italian...",2.0,"Winterfeldtstraße 34, Berlin-Bezirk Tempelhof-...",52.497107,13.355709,4.6,"Monday: 5:00 – 11:00 PM, Tuesday: 5:00 – 11:00...",1980,['We ate there today and I got a season dish -...,"[5, 5, 4]",98,ChIJrcPw105QqEcR_0-GmjViOMk,4,2024-08-16 20:11:26.580097,11:01:52
98,ChIJrT9iSUlQqEcRzKFA6wXFOJU,Ristorante Centro Sud,Mellow fixture with outdoor seating offering I...,2.0,"Bülowstraße 10, Berlin-Bezirk Tempelhof-Schöne...",52.498499,13.357536,4.3,"Monday: 12:00 – 10:00 PM, Tuesday: 12:00 – 10:...",326,"['Tonight, me and my family went to this resta...","[1, 5, 5]",99,ChIJrT9iSUlQqEcRzKFA6wXFOJU,4,2023-12-20 20:11:26.580101,10:44:32


In [62]:
# Set the range for the number of rows per user (e.g., between 1 and 10 rows per user)
min_rows_per_user = 1
max_rows_per_user = 6

data = []
# Loop through each user_id and generate random rows
place_ids = user_restaurants['Place ID'].to_list()

for user_id in user_ids:
    # Randomly choose how many rows this user will have (between min_rows_per_user and max_rows_per_user)
    num_rows = random.randint(min_rows_per_user, max_rows_per_user)
    
    # Randomly select unique place_ids for this user
    selected_place_ids = random.sample(list(place_ids), num_rows)
    
    # Append the generated rows to the data list
    for place_id in selected_place_ids:
        data.append({'user_id': user_id, 'place_id': place_id})

# Convert the list of dictionaries into a pandas DataFrame
random_rows_table = pd.DataFrame(data)

# Display the generated table
random_rows_table.rename(columns={'place_id': 'Place ID'}, inplace=True)

In [84]:
# using the random_rows_table, merge with the user_restaurants table and randomly fill nan values 
user_ratings = np.random.randint(1, 6, 356)

random_users_table = random_rows_table.merge(user_restaurants, on='Place ID', how='inner')

random_users_table['user_ratings'] = user_ratings
visit_dates = [datetime.datetime.now() - datetime.timedelta(days=random.randint(1, 365)) for _ in range(356)]
visit_times = [random_time_between() for _ in range(356)]
random_users_table['visit_date'] = visit_dates
random_users_table['visit_time'] = visit_times



In [86]:

random_users_table.sort_values(['user_id', 'Place ID']).to_csv('random_user_historical_data.csv', index=False)

In [11]:
import pandas as pd 
random_users_table = pd.read_csv('random_user_historical_data.csv')
random_users_table['Price Level'].unique()

array([nan,  2.,  1.,  3.,  4.])

In [20]:
random_users_table

,user_id,Place ID,Name,Summary,Price Level,Address,Latitude,Longitude,Overall Rating,Opening Hours,Total Reviews,Reviews,Review Ratings,user_ratings,visit_date,visit_time
0,1,ChIJLT1gFNtRqEcRUIdXWAGRLB8,Restaurant Austernbank,"Upscale seafood plates presented in a grand, s...",NaN,"Behrenstraße 42, Berlin",52.515929,13.391821,4.3,"Monday: Closed, Tuesday: 5:00 PM – 1:00 AM, We...",323,"[""Was ok, I guess...\nI'd say everything was a...","[4, 5, 2]",1,2023-12-10 20:34:25.536555,14:02:46
1,1,ChIJOeDFuFtOqEcRE8S2p2Wcu-c,Trattoria Libau,Unassuming operation with an old-school vibe d...,2.0,"Libauer Straße 10, Berlin-Bezirk Friedrichshai...",52.508665,13.452836,4.5,"Monday: 4:00 PM – 12:00 AM, Tuesday: 4:00 PM –...",1116,['Good place for a quick Italian food. The piz...,"[4, 5, 5]",5,2024-07-27 20:34:25.536559,17:40:19
2,1,ChIJQUz4iklQqEcRoR2HY1JwkQE,Fadi Food,Unpretentious restaurant serving shawarma wrap...,1.0,"Potsdamer Straße 117, Berlin-Bezirk Tempelhof-...",52.498779,13.362772,4.2,"Monday: 7:00 AM – 11:00 PM, Tuesday: 7:00 AM –...",840,['Amazingly decent food that tastes exactly li...,"[5, 5, 5]",4,2023-09-24 20:34:25.536540,18:57:03
3,1,ChIJrfRNQslPqEcRPuCFqF0-DMI,St. Bart,Bar & kitchen with simple tiled décor plating ...,2.0,"Graefestraße 71, Berlin-Bezirk Friedrichshain-...",52.491920,13.417182,4.5,"Monday: 6:00 – 11:30 PM, Tuesday: 6:00 – 11:30...",664,['Really tasty meal and superb experience! All...,"[5, 4, 3]",3,2024-06-25 20:34:25.536563,10:54:32
4,2,ChIJ4Vof-khQqEcR1CPHIeU2ud4,Nafis Restaurant,"This small, modest Persian spot with tradition...",2.0,"Winterfeldtstraße 11, Berlin-Bezirk Tempelhof-...",52.496140,13.359165,4.3,"Monday: Closed, Tuesday: 11:30 AM – 10:30 PM, ...",639,"[""Super fair prices for tasty, authentic Irani...","[5, 4, 5]",5,2023-10-07 20:34:25.536566,13:08:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,99,ChIJ_8i7pJFPqEcRxmFgAssALik,Taverna Olympia,Mediterranean standards like meze & gyros in a...,2.0,"Emser Straße 73, Berlin-Bezirk Neukölln",52.467257,13.421398,4.6,"Monday: 4:30 – 11:00 PM, Tuesday: Closed, Wedn...",668,['Excellent food and superb service. Very conv...,"[5, 5, 4]",4,2023-10-16 20:34:25.537566,10:59:57
352,99,ChIJhYcCg_RRqEcR2BHrK-3C_nI,Café & Bistro freysinn,"Salads, soups & pasta dishes, plus vegetarian ...",2.0,"Jasmunder Straße 5, Berlin",52.539842,13.388170,4.3,"Monday: 8:00 AM – 3:00 PM, Tuesday: 8:00 AM – ...",194,"['Delicious food, you can go every day and you...","[5, 2, 5]",2,2023-11-09 20:34:25.537572,19:49:43
353,100,ChIJFRjYpfpRqEcRzQcG_2x5Hvk,The Tree,"Szechuan rice bowls, noodle soups & small plat...",2.0,"Brunnenstraße 167, Berlin",52.533774,13.398074,4.6,"Monday: 12:00 – 3:00 PM, 5:30 – 10:00 PM, Tues...",1004,"['The interior design is very creative, with a...","[4, 5, 4]",5,2024-09-07 20:34:25.537580,20:55:47
354,100,ChIJN1PxiuBTqEcRFCOAciXXaCc,Baba đu Restaurant,Hip eatery in an industrial-style space with o...,2.0,"Breite Straße 50, Berlin",52.572023,13.414880,4.8,"Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 AM...",871,"[""Nice selection of dishes. We initially came ...","[4, 5, 5]",1,2023-10-06 20:34:25.537583,20:30:34


In [19]:
# create a user table using random numbers with the following columns: user_id, age, gender, preferred cuisine, preferred price range
import numpy as np 
import random 

user_ids = np.arange(1, 101)
ages = np.random.randint(18, 65, 100)
gender = np.random.choice(['M', 'F'],100) # 1 for female, 0 for male 
# Define the list of available cuisines
cuisine_options = ['Italian', 'Chinese', 'Mexican', 'American', 'Japanese', 'Korean', 'Persian', 'Greek', 'German']
preferred_price_range = np.random.choice(['', 1, 2, 3, 4], 100)
# Generate a list where each user gets 1 to 3 random preferred cuisines
preferred_cuisine = [
    np.random.choice(cuisine_options, size=random.randint(1, 3), replace=False).tolist()
    for _ in range(100)]


user_data = {
    'user_id': user_ids,
    'age': ages,
    'gender': gender, 
    'preferred_cuisine': preferred_cuisine,
    'preferred_price_range': preferred_price_range
}

user_table = pd.DataFrame(user_data)
user_table.to_csv('user_profile_data.csv', index=False)


In [21]:
user_table

,user_id,age,gender,preferred_cuisine,preferred_price_range
0,1,48,M,[American],3
1,2,28,F,"[Persian, German, Korean]",4
2,3,25,M,"[Greek, Chinese]",3
3,4,40,M,"[German, Korean, Italian]",1
4,5,59,F,[Persian],3
...,...,...,...,...,...
95,96,40,F,"[German, Chinese, American]",
96,97,46,F,"[Italian, Persian, Chinese]",1
97,98,37,F,[Italian],1
98,99,47,F,"[Italian, Persian]",4


In [27]:
# # rearrange columns 
# user_restaurants = user_restaurants[['Place ID', 'Name', 'Summary', 'Price Level', 'Address', 'Latitude','Longitude', 'Overall Rating', 'Opening Hours', 'Total Reviews','Reviews', 'Review Ratings']]
# user_restaurants.to_csv('user_restaurants.csv', index=False)